<a href="https://colab.research.google.com/github/wizard339/article_finder/blob/main/article_finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import urllib.request
import urllib.parse
import argparse
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import time

In [54]:
BASE_URL = 'http://export.arxiv.org/api/query?search_query='
PREFIX = {'Title': 'ti',
          'Author': 'au',
          'Abstract': 'abs',
          'Comment': 'co',
          'Journal Reference': 'jr',
          'Subject Category': 'cat',
          'Report Number': 'rn',
          'ID': 'id',
          'All': 'all'}
TIME_PERIOD = {'Day': datetime.timedelta(1),
               'Week': datetime.timedelta(7),
               'Mounth': datetime.timedelta(30),
               'Year': datetime.timedelta(365)}
CURRENT_DATE = datetime.datetime.now()

# input_keywords = input('Please enter the keywords or search phrases separated by commas: ')
input_keywords = 'reinforcement AND learning'

In [55]:
CURRENT_DATE - TIME_PERIOD['Week']

datetime.datetime(2022, 8, 25, 14, 19, 53, 142867)

In [57]:
def make_query(url=BASE_URL, prefix=PREFIX['All'], start=0, keywords=input_keywords, time_period=TIME_PERIOD['Week']):
    """нужно ввести счетчик и проверку на значение тега updated в entry"""
    # the starting date from which we receive the data
    starting_date = CURRENT_DATE - time_period
    # encoding of the search query
    keywords = urllib.parse.quote_plus(keywords)

    cols = ['updated', 'title', 'summary', 'author', 'link']
    articles = pd.DataFrame(columns=cols)
    start_counter = 0
    while True:
      url = f'{url}{prefix}:{keywords}&start={start_counter}&max_results=200&sortBy=lastUpdatedDate&sortOrder=descending'
      print(url)
      req = urllib.request.Request(url)
      with urllib.request.urlopen(req) as response:
          if response.status==200:
              soup = BeautifulSoup(response, 'xml')
          else:
              raise ValueError('Please, check the correctness of the request')
              break
      date_on_the_page = datetime.datetime(*[int(i) for i in soup.entry.updated.string.split('T')[0].split('-')])
      if date_on_the_page >= starting_date:
        print(soup.entry.updated)
        for tag in soup.find_all('entry'):
            updated_to_df = pd.to_datetime(tag.updated.string.split('T')[0])
            authors_list = ", ".join([n.string for n in tag.find_all('name')])
            link_to_df = tag.find(title='pdf').get('href')
            row_to_concat = pd.DataFrame({'updated': updated_to_df,
                                          'title': tag.title,
                                          'summary': tag.summary,
                                          'author': authors_list,
                                          'link': link_to_df}, index=tag.id)
            articles = pd.concat([articles, row_to_concat])
      else:
        break
      start_counter += 1
      time.sleep(3)

    return articles

In [58]:
articles = make_query()

http://export.arxiv.org/api/query?search_query=all:reinforcement+AND+learning&start=0&max_results=200&sortBy=lastUpdatedDate&sortOrder=descending
<updated>2022-08-31T15:52:02Z</updated>
http://export.arxiv.org/api/query?search_query=all:reinforcement+AND+learning&start=0&max_results=200&sortBy=lastUpdatedDate&sortOrder=descendingall:reinforcement+AND+learning&start=1&max_results=200&sortBy=lastUpdatedDate&sortOrder=descending


HTTPError: ignored

In [47]:
datetime.datetime(*[int(i) for i in soup.entry.updated.string.split('T')[0].split('-')])

datetime.datetime(2022, 8, 31, 0, 0)

In [39]:
datetime.datetime(2022,8,31)

datetime.datetime(2022, 8, 31, 0, 0)

In [29]:
articles.sample(10)

,updated,title,summary,author,link
http://arxiv.org/abs/2205.06212v2,2022-07-20,Contingency-constrained economic dispatch with...,Future power systems will rely heavily on mi...,"Michael Eichelbeck, Hannah Markgraf, Matthias ...",http://arxiv.org/pdf/2205.06212v2
http://arxiv.org/abs/2012.08950v4,2022-08-14,Revocable Deep Reinforcement Learning with Aff...,Graph matching (GM) has been a building bloc...,"Chang Liu, Runzhong Wang, Zetian Jiang, Junchi...",http://arxiv.org/pdf/2012.08950v4
http://arxiv.org/abs/2110.12352v2,2022-07-26,DiffSRL: Learning Dynamical State Representati...,Dynamic state representation learning is an ...,"Sirui Chen, Yunhao Liu, Jialong Li, Shang Wen ...",http://arxiv.org/pdf/2110.12352v2
http://arxiv.org/abs/2102.11075v3,2022-06-29,SENTINEL: Taming Uncertainty with Ensemble-bas...,"In this paper, we consider risk-sensitive se...","Hannes Eriksson, Debabrota Basu, Mina Alibeigi...",http://arxiv.org/pdf/2102.11075v3
http://arxiv.org/abs/2208.06267v1,2022-08-12,Causal Imitation Learning with Unobserved Conf...,One of the common ways children learn is by ...,"Junzhe Zhang, Daniel Kumor, Elias Bareinboim",http://arxiv.org/pdf/2208.06267v1
http://arxiv.org/abs/2009.14108v2,2022-06-28,Align-RUDDER: Learning From Few Demonstrations...,Reinforcement learning algorithms require ma...,"Vihang P. Patil, Markus Hofmarcher, Marius-Con...",http://arxiv.org/pdf/2009.14108v2
http://arxiv.org/abs/2009.09575v2,2022-07-07,Human Engagement Providing Evaluative and Info...,Interactive reinforcement learning proposes ...,"Adam Bignold, Francisco Cruz, Richard Dazeley,...",http://arxiv.org/pdf/2009.09575v2
http://arxiv.org/abs/2208.12125v1,2022-08-25,UAS Navigation in the Real World Using Visual ...,This paper presents a novel end-to-end Unman...,"Yuci Han, Jianli Wei, Alper Yilmaz",http://arxiv.org/pdf/2208.12125v1
http://arxiv.org/abs/2111.01294v2,2022-08-16,Learning to Operate an Electric Vehicle Chargi...,The rapid adoption of electric vehicles (EVs...,"Zuzhao Ye, Yuanqi Gao, Nanpeng Yu",http://arxiv.org/pdf/2111.01294v2
http://arxiv.org/abs/2112.04454v2,2022-07-04,Greedy-based Value Representation for Optimal ...,Due to the representation limitation of the ...,"Lipeng Wan, Zeyang Liu, Xingyu Chen, Han Wang,...",http://arxiv.org/pdf/2112.04454v2


In [30]:
articles.updated.value_counts()

2022-07-07    30
2022-06-28    28
2022-06-21    27
2022-07-01    26
2022-07-13    23
              ..
2022-07-09     5
2022-08-20     5
2022-07-10     5
2022-08-13     4
2022-08-14     3
Name: updated, Length: 72, dtype: int64

In [ ]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, http://arxiv.org/abs/2208.13764v1 to http://arxiv.org/abs/2208.10993v1
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   updated  1000 non-null   datetime64[ns]
 1   title    1000 non-null   object        
 2   summary  1000 non-null   object        
 3   author   1000 non-null   object        
 4   link     1000 non-null   object        
dtypes: datetime64[ns](1), object(4)
memory usage: 46.9+ KB
